In [1]:
# Author: Marco Lugo
# Simple classification model to distinguish between the different shapes (rectangles vs circles)
# Data has 1800 128x128x3 images, with 20% of the data used for validation
# Achieves ~99.70% after 15 epochs

import numpy as np

import tensorflow as tf
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model
from keras.layers import Input
from keras.layers.core import Dropout, Flatten, Dense
from keras.optimizers import Adam

img_size = 128
nb_epochs = 15

Using TensorFlow backend.


In [2]:
train_datagen = ImageDataGenerator(
        rescale=1/255,
        zoom_range=0.25,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True,
        vertical_flip=True)

test_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
        'input/train/images',
        target_size=(img_size, img_size),
        batch_size=32,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        'input/validation/images',
        target_size=(img_size, img_size),
        batch_size=32,
        class_mode='categorical')

Found 1463 images belonging to 2 classes.
Found 337 images belonging to 2 classes.


In [3]:
def build_model(img_size=128):
    inputs = Input((img_size, img_size, 3))

    x = Conv2D(8, (3, 3), activation='relu', padding='same')(inputs)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)

    x = GlobalMaxPooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.1)(x)
    outputs = Dense(2, activation='softmax')(x)

    model = Model(inputs=[inputs], outputs=[outputs])
    return model

model = build_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 8)       224       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 128, 8)       584       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 8)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 64, 16)        1168      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 64, 64, 16)        2320      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 32, 16)        0         
__________

In [4]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit_generator(train_generator, validation_data=validation_generator, epochs=nb_epochs, verbose=1)

Epoch 1/15
46/46 [==============================] - 6s 132ms/step - loss: 0.8650 - acc: 0.4964 - val_loss: 0.6929 - val_acc: 0.5163
Epoch 2/15
46/46 [==============================] - 5s 109ms/step - loss: 0.6906 - acc: 0.5338 - val_loss: 0.6617 - val_acc: 0.5727
Epoch 3/15
46/46 [==============================] - 5s 109ms/step - loss: 0.4920 - acc: 0.7554 - val_loss: 0.1394 - val_acc: 0.9436
Epoch 4/15
46/46 [==============================] - 5s 110ms/step - loss: 0.1692 - acc: 0.9231 - val_loss: 0.0535 - val_acc: 0.9822
Epoch 5/15
46/46 [==============================] - 5s 109ms/step - loss: 0.1289 - acc: 0.9402 - val_loss: 0.0728 - val_acc: 0.9674
Epoch 6/15
46/46 [==============================] - 5s 110ms/step - loss: 0.1566 - acc: 0.9284 - val_loss: 0.0422 - val_acc: 0.9852
Epoch 7/15
46/46 [==============================] - 5s 110ms/step - loss: 0.0720 - acc: 0.9689 - val_loss: 0.0346 - val_acc: 0.9852
Epoch 8/15
46/46 [==============================] - 5s 112ms/step - loss: 0.